In [14]:
import re
import mne
import wfdb
import glob
import pathlib
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import tensorflow_io as tfio
import tensorflow as tf

In [2]:
path = "../data/downloads/sleep_edfx"

records = glob.glob(f"{path}/**/*.edf", recursive=True)

In [3]:
record = records[0]
raw = mne.io.read_raw_edf(record, infer_types=True, stim_channel="Event marker", preload=True)
picks = mne.pick_types(raw.info, eeg=True)
channels = raw.info["ch_names"]
data = raw.get_data(picks=picks)
# if f"{record}.seizures" in seizures:
#     seizure = wfdb.io.rdann(record, extension="seizures")
#     start = seizure.sample[0] / raw.info["sfreq"]
#     finish = seizure.sample[1] / raw.info["sfreq"]
#     raw.set_annotations(mne.Annotations(onset=start, duration=(finish - start), description="seizure"))

Extracting EDF parameters from /workspaces/data/downloads/sleep_edfx/sleep-cassette/SC4001E0-PSG.edf...
EDF file detected
Channel 'EEG Fpz-Cz' recognized as type EEG (renamed to 'Fpz-Cz').
Channel 'EEG Pz-Oz' recognized as type EEG (renamed to 'Pz-Oz').
Channel 'EOG horizontal' recognized as type EOG (renamed to 'horizontal').
Channel 'Resp oro-nasal' recognized as type RESP (renamed to 'oro-nasal').
Channel 'EMG submental' recognized as type EMG (renamed to 'submental').
Channel 'Temp rectal' recognized as type TEMP (renamed to 'rectal').
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7949999  =      0.000 ... 79499.990 secs...


In [18]:
raw_new = raw.crop(tmin=1000, tmax=3000)

In [19]:
raw.get_data().shape

(7, 200001)

In [8]:
tb = pa.Table.from_pydict({
    "A": [1,2,3],
    "B": pa.array([np.random.rand(3,3).tobytes(), np.random.rand(3,3).tobytes(), np.random.rand(3,3).tobytes()], type=pa.binary())
})

pq.write_table(tb, "test.parquet")

In [17]:
ds = tfio.IODataset.from_parquet("test.parquet")

for x in ds:
    print(tf.io.parse_tensor(x[b"B"], out_type=tf.float32))

InvalidArgumentError: {{function_node __wrapped__ParseTensor_device_/job:localhost/replica:0/task:0/device:CPU:0}} Could not parse `serialized` as TensorProto, base64: WCbIqyOMtT9gdfMq+0nSP8vjvjLrJug/JBHPjnbo4z+rzDKARrTgP56V48a7/tY/AOOTm3uNyj8ViSv/0pLnP2yb6XirkN8/ [Op:ParseTensor] name: 

In [33]:
epochs = mne.make_fixed_length_epochs(raw, duration=30, overlap=0)

Not setting metadata
2650 matching events found
No baseline correction applied
0 projection items activated


In [34]:
epochs.to_data_frame()

Using data from preloaded Raw for 2650 events and 3000 original time points ...
0 bad epochs dropped


,time,condition,epoch,Fpz-Cz,Pz-Oz,horizontal,oro-nasal,submental,rectal,Event marker
0,0.00,1,0,5.016850,-2.467399,16.508669,-482.000000,3.552000,37.206452,920.0
1,0.01,1,0,-2.578755,1.467399,16.015873,-479.674851,3.553216,37.205160,920.0
2,0.02,1,0,1.359707,-4.098901,9.609524,-477.059581,3.554425,37.203907,920.0
3,0.03,1,0,-2.391209,-2.659341,1.724786,-474.153663,3.555626,37.202694,920.0
4,0.04,1,0,-5.204396,-0.739927,7.638339,-470.956747,3.556817,37.201522,920.0
...,...,...,...,...,...,...,...,...,...,...
7949995,29.95,1,2649,11.299634,-6.882051,82.050549,-489.304581,3.545879,37.213498,898.0
7949996,29.96,1,2649,3.610256,-8.609524,62.338706,-488.416044,3.547104,37.212011,898.0
7949997,29.97,1,2649,5.298168,-3.043223,69.730647,-487.242620,3.548330,37.210563,898.0
7949998,29.98,1,2649,-4.641758,-3.715018,80.572161,-485.782923,3.549555,37.209153,898.0


In [26]:
raw.to_data_frame()

,time,Fpz-Cz,Pz-Oz,horizontal,oro-nasal,submental,rectal,Event marker
0,0.00,5.016850,-2.467399,16.508669,-482.000000,3.552000,37.206452,920.0
1,0.01,-2.578755,1.467399,16.015873,-479.674851,3.553216,37.205160,920.0
2,0.02,1.359707,-4.098901,9.609524,-477.059581,3.554425,37.203907,920.0
3,0.03,-2.391209,-2.659341,1.724786,-474.153663,3.555626,37.202694,920.0
4,0.04,-5.204396,-0.739927,7.638339,-470.956747,3.556817,37.201522,920.0
...,...,...,...,...,...,...,...,...
7949995,79499.95,11.299634,-6.882051,82.050549,-489.304581,3.545879,37.213498,898.0
7949996,79499.96,3.610256,-8.609524,62.338706,-488.416044,3.547104,37.212011,898.0
7949997,79499.97,5.298168,-3.043223,69.730647,-487.242620,3.548330,37.210563,898.0
7949998,79499.98,-4.641758,-3.715018,80.572161,-485.782923,3.549555,37.209153,898.0


In [25]:
epochs.to_data_frame()

Using data from preloaded Raw for 2650 events and 3000 original time points ...
0 bad epochs dropped


,time,condition,epoch,Fpz-Cz,Pz-Oz,horizontal,oro-nasal,submental,rectal,Event marker
0,0.00,1,0,5.016850,-2.467399,16.508669,-482.000000,3.552000,37.206452,920.0
1,0.01,1,0,-2.578755,1.467399,16.015873,-479.674851,3.553216,37.205160,920.0
2,0.02,1,0,1.359707,-4.098901,9.609524,-477.059581,3.554425,37.203907,920.0
3,0.03,1,0,-2.391209,-2.659341,1.724786,-474.153663,3.555626,37.202694,920.0
4,0.04,1,0,-5.204396,-0.739927,7.638339,-470.956747,3.556817,37.201522,920.0
...,...,...,...,...,...,...,...,...,...,...
7949995,29.95,1,2649,11.299634,-6.882051,82.050549,-489.304581,3.545879,37.213498,898.0
7949996,29.96,1,2649,3.610256,-8.609524,62.338706,-488.416044,3.547104,37.212011,898.0
7949997,29.97,1,2649,5.298168,-3.043223,69.730647,-487.242620,3.548330,37.210563,898.0
7949998,29.98,1,2649,-4.641758,-3.715018,80.572161,-485.782923,3.549555,37.209153,898.0


In [13]:
epochs.get_data().shape

Using data from preloaded Raw for 2650 events and 3000 original time points ...


(2650, 7, 0)

In [32]:
for pick in picks:
    print(channels[pick])

EEG Fpz-Cz
EEG Pz-Oz
EOG horizontal
Resp oro-nasal
EMG submental
Temp rectal


In [22]:
raw.info["ch_names"]

['Fpz-Cz',
 'Pz-Oz',
 'horizontal',
 'oro-nasal',
 'submental',
 'rectal',
 'Event marker']

In [29]:
montage = mne.channels.make_standard_montage('standard_1020')
ch_pos = montage.get_positions()["ch_pos"]
positions = {key.upper(): val for key, val in ch_pos.items()}

In [33]:
channels = raw.info["ch_names"]
sources = np.zeros((len(channels), 3))
targets = np.zeros((len(channels), 3))
for idx, channel in enumerate(channels):
    electrodes = channel.split("-")
    sources[idx] = positions[electrodes[0]]
    targets[idx] = positions[electrodes[1]]
    print(positions[electrodes[0]])
    print(positions[electrodes[1]])
    print("-------------------")

[-0.0294367  0.0839171 -0.00699  ]
[-0.0702629  0.0424743 -0.01142  ]
-------------------
[-0.0702629  0.0424743 -0.01142  ]
[-0.0841611 -0.0160187 -0.009346 ]
-------------------
[-0.0841611 -0.0160187 -0.009346 ]
[-0.0724343 -0.0734527 -0.002487 ]
-------------------
[-0.0724343 -0.0734527 -0.002487 ]
[-0.0294134 -0.112449   0.008839 ]
-------------------
[-0.0294367  0.0839171 -0.00699  ]
[-0.0502438  0.0531112  0.042192 ]
-------------------
[-0.0502438  0.0531112  0.042192 ]
[-0.0653581 -0.0116317  0.064358 ]
-------------------
[-0.0653581 -0.0116317  0.064358 ]
[-0.0530073 -0.0787878  0.05594  ]
-------------------
[-0.0530073 -0.0787878  0.05594  ]
[-0.0294134 -0.112449   0.008839 ]
-------------------
[ 0.0298723  0.0848959 -0.00708  ]
[0.0518362 0.0543048 0.040814 ]
-------------------
[0.0518362 0.0543048 0.040814 ]
[ 0.0671179 -0.0109003  0.06358  ]
-------------------
[ 0.0671179 -0.0109003  0.06358  ]
[ 0.0556667 -0.0785602  0.056561 ]
-------------------
[ 0.0556667 -0.0

In [28]:
for channel in raw.info["ch_names"]:
    x = channel.split("-")[0]
    y = channel.split("-")[1]
    print(ch_pos[x])
    print(ch_pos[y])
    print("------------------")

[-0.0294367  0.0839171 -0.00699  ]
[-0.0702629  0.0424743 -0.01142  ]
------------------
[-0.0702629  0.0424743 -0.01142  ]
[-0.0841611 -0.0160187 -0.009346 ]
------------------
[-0.0841611 -0.0160187 -0.009346 ]
[-0.0724343 -0.0734527 -0.002487 ]
------------------
[-0.0724343 -0.0734527 -0.002487 ]
[-0.0294134 -0.112449   0.008839 ]
------------------
[-0.0294367  0.0839171 -0.00699  ]
[-0.0502438  0.0531112  0.042192 ]
------------------
[-0.0502438  0.0531112  0.042192 ]
[-0.0653581 -0.0116317  0.064358 ]
------------------
[-0.0653581 -0.0116317  0.064358 ]
[-0.0530073 -0.0787878  0.05594  ]
------------------
[-0.0530073 -0.0787878  0.05594  ]
[-0.0294134 -0.112449   0.008839 ]
------------------
[ 0.0298723  0.0848959 -0.00708  ]
[0.0518362 0.0543048 0.040814 ]
------------------
[0.0518362 0.0543048 0.040814 ]
[ 0.0671179 -0.0109003  0.06358  ]
------------------
[ 0.0671179 -0.0109003  0.06358  ]
[ 0.0556667 -0.0785602  0.056561 ]
------------------
[ 0.0556667 -0.0785602  0.0

In [69]:
[channel.split("-")[0] for channel in raw.info["ch_names"]]

['FP1',
 'F7',
 'T7',
 'P7',
 'FP1',
 'F3',
 'C3',
 'P3',
 'FP2',
 'F4',
 'C4',
 'P4',
 'FP2',
 'F8',
 'P8',
 'FZ',
 'CZ',
 'P7',
 'T7',
 'FT9',
 'FT10']

In [67]:
anodes = [channel.split("-")[0] for channel in raw.info["ch_names"]]
cathodes = [channel.split("-")[1] for channel in raw.info["ch_names"]]
mne.set_bipolar_reference(raw, anode=anodes, cathode=cathodes)

ValueError: There is already a channel named "FP1-F7", please specify a different name for the bipolar channel using the ch_name parameter.

In [68]:
raw.rename_channels(lambda ch_name: ch_name.split("-")[0])

ValueError: New channel names are not unique, renaming failed

In [35]:
mapping = {
    'FP1-F7': 'AF7',
    'F7-T7': 'FT7',
    'T7-P7': 'TP7',
    'P7-O1': 'PO7',
    'FP1-F3': 'AF3',
    'F3-C3': 'FC3',
    'C3-P3': 'CP3',
    'P3-O1': 'PO3',
    'FP2-F4': 'AF4',
    'F4-C4': 'FC4',
    'C4-P4': 'CP4',
    'P4-O2': 'PO4',
    'FP2-F8': 'AF8',
    'F8-T8': 'FT8',
    'T8-P8-0': 'TP8',
    'P8-O2': 'PO8',
    'FZ-CZ': 'FCz',
    'CZ-PZ': 'CPz',
    'P7-T7': 'TP7',
    'T7-FT9': '???',
    'FT9-FT10': '???',
    'FT10-T8': '???',
    'T8-P8-1': 'TP8'
}

('FP', '1', 'F', '7')
('F', '7', 'T', '7')
('T', '7', 'P', '7')
('P', '7', 'O', '1')
('FP', '1', 'F', '3')
('F', '3', 'C', '3')
('C', '3', 'P', '3')
('P', '3', 'O', '1')
('FP', '2', 'F', '4')
('F', '4', 'C', '4')
('C', '4', 'P', '4')
('P', '4', 'O', '2')
('FP', '2', 'F', '8')
('F', '8', 'T', '8')
('T', '8', 'P', '8')
('P', '8', 'O', '2')
('FZ', '', 'CZ', '')
('CZ', '', 'PZ', '')
('P', '7', 'T', '7')
('T', '7', 'FT', '9')
('FT', '9', 'FT', '10')
('FT', '10', 'T', '8')
('T', '8', 'P', '8')


{'FP1-F7': 'FP1',
 'F7-T7': 'F7',
 'T7-P7': 'T7',
 'P7-O1': 'P7',
 'FP1-F3': 'FP1',
 'F3-C3': 'F3',
 'C3-P3': 'C3',
 'P3-O1': 'P3',
 'FP2-F4': 'FP2',
 'F4-C4': 'F4',
 'C4-P4': 'C4',
 'P4-O2': 'P4',
 'FP2-F8': 'FP2',
 'F8-T8': 'F8',
 'T8-P8-0': 'T8',
 'P8-O2': 'P8',
 'FZ-CZ': 'FZ',
 'CZ-PZ': 'CZ',
 'P7-T7': 'P7',
 'T7-FT9': 'T7',
 'FT9-FT10': 'FT9',
 'FT10-T8': 'FT10',
 'T8-P8-1': 'T8'}

In [73]:
raw.set_montage("standard_1020", match_case=False, on_missing="warn")

/tmp/ipykernel_4364/2381065304.py:1: RuntimeWarning: DigMontage is only a subset of info. There are 21 channel positions not present in the DigMontage. The channels missing from the montage are:

['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  raw.set_montage("standard_1020", match_case=False, on_missing="warn")


Measurement date,"November 06, 2076 14:43:12 GMT"
Experimenter,Unknown
Participant,Surrogate
Digitized points,3 points
Good channels,21 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,256.00 Hz
Highpass,0.00 Hz
Lowpass,128.00 Hz


In [84]:
sampling_freq = 100

times = np.linspace(0, 1, sampling_freq, endpoint=False)
sine = np.sin(20 * np.pi * times)
cosine = np.cos(10 * np.pi * times)
data = np.array([sine, cosine])

info = mne.create_info(
    ch_names=["10 Hz sine", "5 Hz cosine"], ch_types=["misc"] * 2, sfreq=sampling_freq
)

test_raw = mne.io.RawArray(data, info)

Creating RawArray with float64 data, n_channels=2, n_times=100
    Range : 0 ... 99 =      0.000 ...     0.990 secs
Ready.


In [100]:
x, y = raw["C3-P3"]
print(x[0])
print(y)

[-1.54334554e-05  1.95360195e-07  1.95360195e-07 ...  5.86080586e-07
 -1.75824176e-06  3.32112332e-06]
[0.00000000e+00 3.90625000e-03 7.81250000e-03 ... 3.59998828e+03
 3.59999219e+03 3.59999609e+03]


In [96]:
x = 3.59999609e+03

In [95]:
x, y = test_raw["10 Hz sine"]
print(x)
print(y)

[[ 0.00000000e+00  5.87785252e-01  9.51056516e-01  9.51056516e-01
   5.87785252e-01  1.22464680e-16 -5.87785252e-01 -9.51056516e-01
  -9.51056516e-01 -5.87785252e-01 -2.44929360e-16  5.87785252e-01
   9.51056516e-01  9.51056516e-01  5.87785252e-01  3.67394040e-16
  -5.87785252e-01 -9.51056516e-01 -9.51056516e-01 -5.87785252e-01
  -4.89858720e-16  5.87785252e-01  9.51056516e-01  9.51056516e-01
   5.87785252e-01  6.12323400e-16 -5.87785252e-01 -9.51056516e-01
  -9.51056516e-01 -5.87785252e-01 -7.34788079e-16  5.87785252e-01
   9.51056516e-01  9.51056516e-01  5.87785252e-01 -2.69546092e-15
  -5.87785252e-01 -9.51056516e-01 -9.51056516e-01 -5.87785252e-01
  -9.79717439e-16  5.87785252e-01  9.51056516e-01  9.51056516e-01
   5.87785252e-01  1.10218212e-15 -5.87785252e-01 -9.51056516e-01
  -9.51056516e-01 -5.87785252e-01 -1.22464680e-15  5.87785252e-01
   9.51056516e-01  9.51056516e-01  5.87785252e-01 -2.20560220e-15
  -5.87785252e-01 -9.51056516e-01 -9.51056516e-01 -5.87785252e-01
  -1.46957

In [75]:
raw.plot_sensors()

RuntimeError: No valid channel positions found

In [12]:
x = mne.make_fixed_length_epochs(raw, 7)

Not setting metadata
514 matching events found
No baseline correction applied
0 projection items activated


In [13]:
print(raw.get_data().shape)

(23, 921600)


In [15]:
raw.plot_sensors()

RuntimeError: No valid channel positions found

In [14]:
print(x.get_data().shape)

Loading data for 514 events and 1792 original time points ...
0 bad epochs dropped
(514, 23, 1792)
